<a href="https://colab.research.google.com/github/FKz11/NLP/blob/main/lesson_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson-13

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует

Данные

In [1]:
!pip install -qq datasets

In [2]:
from datasets import load_dataset

dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


  0%|          | 0/3 [00:00<?, ?it/s]

Так как эта модель уже обучена на этих данных, то нам нужно взять только test данные

Так же это модель очень тяжелая, даже для генерации, не говоря уже про обучение, поэтому возьмём только 256 примеров для анализа.

In [3]:
test_size = 256

In [4]:
data = dataset['test'][:test_size]

In [5]:
data_x = data['text']
data_y = data['summary']
len(data_x)

256

### Загружаем предобученную модель

In [6]:
!pip install -qq transformers[SentencePiece]

In [7]:
from transformers import AutoTokenizer, MBartForConditionalGeneration #AutoModel

model_name = "IlyaGusev/mbart_ru_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).cuda()

### Predict

In [8]:
input_ids = tokenizer.prepare_seq2seq_batch(
    data_x,
    src_lang="ru_XX", 
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=600
)["input_ids"].cuda()

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3668: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [9]:
input = []
batch_size=16
i=0
for _ in range(len(input_ids)//batch_size):
  input.append(input_ids[i:i+batch_size])
  i+=batch_size

In [10]:
import torch
from tqdm import tqdm

In [11]:
output_ids = []
for ids in tqdm(input):
    output_ids.append(model.generate(
    input_ids=ids,
    max_length=162,
    no_repeat_ngram_size=3,
    num_beams=5,
    top_k=0))

100%|██████████| 16/16 [04:22<00:00, 16.41s/it]


In [12]:
torch.save(output_ids, 'tensor.pt')

*** checkpoint ***

In [11]:
output_ids = torch.load('tensor.pt')

In [12]:
output = []
for i in output_ids:
  for j in i:
    output.append(tokenizer.decode(j, skip_special_tokens=True, clean_up_tokenization_spaces=False))
output[0]

'В NASA назвали названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта.'

In [13]:
for i in range(10):
  print(f'True: {data_y[i]}')
  print(f'Predict: {output[i]}')
  print()

True: В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.
Predict: В NASA назвали названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта.

True: 25 и 26 февраля в Кремлевском дворце съездов праздновали Сагаалган — Восточный Новый год. Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.
Predict: В Москве состоялось праздничное шоу с участием более 300 артистов из одного региона. Зрителям рассказали, что Республике Бу

Генерации очень даже хорошие

### Метрики

In [14]:
!pip install -qq rouge

In [15]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

In [16]:
rouge = Rouge()
print(f'Count: {len(output)}')
print(f'BLEU: {corpus_bleu([[i.lower()] for i in data_y], [i.lower() for i in output])}')
print(f'ROUGE: {rouge.get_scores([i.lower() for i in data_y], [i.lower() for i in output], avg=True)}')

Count: 256
BLEU: 0.42411350328424513
ROUGE: {'rouge-1': {'r': 0.25942755302697973, 'p': 0.30898668968549853, 'f': 0.2748373082727381}, 'rouge-2': {'r': 0.11030329406925359, 'p': 0.13513526997718617, 'f': 0.11767730473852774}, 'rouge-l': {'r': 0.23973968638151563, 'p': 0.28620135217384685, 'f': 0.25419879406629364}}


Метрики тоже не плохие